# Data Preprocessing

In [11]:
# importing the libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [12]:
# importing the datasets.
# i have already downloaded the datasets from the following links:
# https://www.kaggle.com/datasets/kapturovalexander/ign-games-from-best-to-worst/data
# https://www.kaggle.com/datasets/hibrahimag1/top-1000-twitch-streamers-data-may-2024
# https://www.kaggle.com/datasets/rankirsh/evolution-of-top-games-on-twitch

ratings = pd.read_csv(r'C:\!Projects\games_analysis\Data\IGN_games_ratings.csv')
ratings_data = ratings.copy()
streamers = pd.read_csv(r'C:\!Projects\games_analysis\Data\Twitch_streamers_data.csv')
streamers_data = streamers.copy()
games = pd.read_csv(r'C:\!Projects\games_analysis\Data\twitch_games_data.csv', encoding='latin1')
games_data = games.copy()

# - quick view of the datasets

In [5]:
ratings_data.head()

,title,score,score_phrase,platform,genre,release_year,release_month,release_day
0,Checkered Flag,10.0,Masterpiece,Lynx,Racing,1999,7,6
1,Chrono Trigger,10.0,Masterpiece,Wii,"Action, RPG",2011,5,25
2,Dragon Warrior III,10.0,Masterpiece,Game Boy Color,RPG,2001,7,20
3,Grand Theft Auto IV,10.0,Masterpiece,Xbox 360,"Action, Adventure",2008,4,25
4,Grand Theft Auto IV,10.0,Masterpiece,PlayStation 3,"Action, Adventure",2008,4,25


In [6]:
streamers_data.head()

,RANK,NAME,LANGUAGE,TYPE,MOST_STREAMED_GAME,2ND_MOST_STREAMED_GAME,AVERAGE_STREAM_DURATION,FOLLOWERS_GAINED_PER_STREAM,AVG_VIEWERS_PER_STREAM,AVG_GAMES_PER_STREAM,TOTAL_TIME_STREAMED,TOTAL_FOLLOWERS,TOTAL_VIEWS,TOTAL_GAMES_STREAMED,ACTIVE_DAYS_PER_WEEK,MOST_ACTIVE_DAY,DAY_WITH_MOST_FOLLOWERS_GAINED
0,1,kaicenat,English,personality,Just Chatting,I'm Only Sleeping,7.6,18.405,15.852,2.3,4.698,10600000,9150000,194,3.6,Friday,Saturday
1,2,jynxzi,English,personality,Tom Clancy's Rainbow Six Siege,NBA 2K20,5.4,3.386,1.145,1.2,8.407,5760000,1950000,54,5.6,Tuesday,Sunday
2,3,caedrel,English,personality,League of Legends,I'm Only Sleeping,6.3,689.000,12.331,1.3,6.728,797000,14200000,111,2.8,Thursday,Sunday
3,4,caseoh_,English,personality,NBA 2K23,Just Chatting,4.6,7.185,0.000,3.6,2.554,4220000,53,385,6.2,Friday,Monday
4,5,ibai,Spanish,personality,Just Chatting,League of Legends,4.1,8.289,190.714,1.5,6.865,15600000,359000000,149,4.3,Wednesday,Saturday


In [8]:
games_data.head()

,Rank,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_viewer_ratio
0,1,League of Legends,1,2016,94377226,1362044,530270,2903,129172,127021,1833,69.29
1,2,Counter-Strike: Global Offensive,1,2016,47832863,830105,372654,2197,120849,64378,1117,57.62
2,3,Dota 2,1,2016,45185893,433397,315083,1100,44074,60815,583,104.26
3,4,Hearthstone,1,2016,39936159,235903,131357,517,36170,53749,317,169.29
4,5,Call of Duty: Black Ops III,1,2016,16153057,1151578,71639,3620,214054,21740,1549,14.03


In [15]:
# starting with the games rating dataset
ratings_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18625 entries, 0 to 18624
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          18625 non-null  object 
 1   score          18625 non-null  float64
 2   score_phrase   18625 non-null  object 
 3   platform       18625 non-null  object 
 4   genre          18589 non-null  object 
 5   release_year   18625 non-null  int64  
 6   release_month  18625 non-null  int64  
 7   release_day    18625 non-null  int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 1.1+ MB


In [17]:
ratings_data.isnull().sum()

title             0
score             0
score_phrase      0
platform          0
genre            36
release_year      0
release_month     0
release_day       0
dtype: int64

In [20]:
ratings_data['genre'].unique()

array(['Racing', 'Action, RPG', 'RPG', 'Action, Adventure', 'Action',
       'Fighting, Action', 'Adventure', 'Puzzle, Adventure',
       'Sports, Action', 'Fighting', 'Puzzle', 'Platformer', 'Strategy',
       'Sports', 'Racing, Simulation', 'Shooter', 'Compilation',
       'Racing, Action', 'Simulation', 'Strategy, RPG', 'Music',
       'Puzzle, Action', 'Party', 'Music, Action', 'Sports, Simulation',
       'Compilation, Compilation', 'Card', 'Flight, Action', 'Hardware',
       'Shooter, RPG', 'Action, Simulation', 'Wrestling',
       'Platformer, Action', 'Puzzle, RPG', 'Trivia', 'Board',
       'Productivity', nan, 'Action, Compilation', 'Card, Battle',
       'Music, Editor', 'Sports, Racing', 'Flight, Simulation', 'Other',
       'Platformer, Adventure', 'Adventure, Compilation', 'Other, Action',
       'Flight', 'Card, RPG', 'Pinball', 'Educational, Action',
       'RPG, Simulation', 'Adventure, Episodic', 'Shooter, First-Person',
       'Music, Adventure', 'Casino', 'Virtual 

In [21]:
ratings_data.fillna('Not known', inplace=True)
ratings_data.isnull().sum()

title            0
score            0
score_phrase     0
platform         0
genre            0
release_year     0
release_month    0
release_day      0
dtype: int64

In [22]:
ratings_data.tail(20)

,title,score,score_phrase,platform,genre,release_year,release_month,release_day
18605,Kidz Sports: Basketball,1.0,Unbearable,Wii,Sports,2008,2,27
18606,Kidz Sports: Ice Hockey,1.0,Unbearable,Wii,Sports,2008,2,26
18607,Kidz Sports: International Soccer,1.0,Unbearable,Wii,Sports,2008,2,29
18608,London Taxi: Rush Hour,1.0,Unbearable,Wii,Racing,2008,2,29
18609,Mega Man 2,1.0,Unbearable,iPhone,Action,2009,3,27
18610,Monster Trux Arenas -- Special Edition,1.0,Unbearable,Wii,Racing,2008,2,6
18611,Motocross Championship,1.0,Unbearable,Sega 32X,Racing,2008,11,20
18612,NFL Blitz 20-03,1.0,Unbearable,Game Boy Advance,"Sports, Action",2002,8,13
18613,NRA Varmint Hunter,1.0,Unbearable,PC,Shooter,2005,1,28
18614,Offroad Extreme -- Special Edition,1.0,Unbearable,Wii,Racing,2007,12,10


In [32]:
# checking for duplicates
duplicated_rows = ratings_data[ratings_data.duplicated(keep=False)]
duplicated_rows

,title,score,score_phrase,platform,genre,release_year,release_month,release_day


In [30]:
ratings_data.drop_duplicates(inplace=True)
ratings_data.duplicated().sum()

0

In [34]:
ratings_data['release_year'].min()

1970